### Installation
Install the packages required for executing this notebook.

In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed. The KFP SDK version should be >=2

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.3.0
google-cloud-aiplatform==1.35.0
google_cloud_pipeline_components version: 2.4.1


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip

#### Pipeline Configurations

In [3]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "data-engineering-assignment-1"
# The region that this pipeline runs in
REGION = "europe-west1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://tmp_data_engineering_assignment_group_7"

#### Create the Components from Component Specifications 

In [5]:
@dsl.container_component
def downloading(project_id: str, data_repo: str, data_file_name: str, dataset_path: Output[Artifact]): 

    return dsl.ContainerSpec(
        image='europe-west1-docker.pkg.dev/data-engineering-assignment-1/docker-image-repo/yolov8_downloader:release-0.1.5',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id', project_id,'--data_bucket', data_repo,'--file_name', data_file_name, '--dataset_path', dataset_path.path])


@dsl.container_component
def training(project_id: str, dataset_path: Input[Artifact], model_repo: str):

    return dsl.ContainerSpec(
        image='europe-west1-docker.pkg.dev/data-engineering-assignment-1/docker-image-repo/yolov8_trainer:release-0.1.5',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id', project_id, '--dataset_path', dataset_path.path, '--model_repo', model_repo])

#### Define the Pipeline

In [6]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="dual-component-pipeline")
def pipeline(project_id: str, data_bucket: str, model_repo: str, data_file_name: str):
    
    # The first step
    download_op = downloading(project_id=project_id, data_repo=data_bucket, data_file_name=data_file_name)
    
    # The second step
    training_op = training(project_id=project_id, dataset_path=download_op.outputs['dataset_path'], model_repo=model_repo)
    

#### Compile the Pipeline

In [7]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='yolov8-training-pipeline.yaml')

#### Run the Pipeline

In [8]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)


job = aip.PipelineJob(
    display_name="yolov8-training-pipeline",
    template_path="yolov8-training-pipeline.yaml",
    enable_caching=False,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'project_id': PROJECT_ID,
        'data_bucket': 'data_repo2023', # makesure to use your data bucket name 
        'model_repo':'model_repo2023', # makesure to use your model bucket name 
        'data_file_name': 'obj_det_dataset.zip'
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/106406165246/locations/europe-west1/pipelineJobs/dual-component-pipeline-20231013090806
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/106406165246/locations/europe-west1/pipelineJobs/dual-component-pipeline-20231013090806')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west1/pipelines/runs/dual-component-pipeline-20231013090806?project=106406165246
PipelineJob projects/106406165246/locations/europe-west1/pipelineJobs/dual-component-pipeline-20231013090806 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/106406165246/locations/europe-west1/pipelineJobs/dual-component-pipeline-20231013090806 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/106406165246/locations/europe-west1/pipelineJobs/dual-component-pipeline-20231013090806 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob proje

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [training].; Job (project_id = data-engineering-assignment-1, job_id = 1733883657514385408) is failed due to the above error.; Failed to handle the job: {project_number = 106406165246, job_id = 1733883657514385408}"
